In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

from utils.utils_chain import Account, WrapperAddress
from utils.utils_tx import endpoint_call
from multiversx_sdk import ProxyNetworkProvider

proxy = ProxyNetworkProvider("http://localhost:8085")
SIMULATOR_URL = "http://localhost:8085"
GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"

setup

In [ ]:
import json
# generate 20 blocks to pass an epoch and the smart contract deploys to be enabled
proxy.do_post(f"{GENERATE_BLOCKS_URL}/20", {})

# set state for wrapped egld contract and system account on shard 1
# load JSON data from the file
with open(Path().home() / "Documents/temp/accounts-state.json", "r") as file:
    json_data = json.load(file)

proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", json_data)

In [ ]:
proxy.do_post(f"{GENERATE_BLOCKS_URL}/5", {})

In [ ]:
contract_address = WrapperAddress("erd1qqqqqqqqqqqqqpgqp8v47r80n2see9a2g77j3qm56gyz279aa7aq460szc")

second_owner = Account(pem_file="~/Documents/sh2.pem")
second_owner.address = WrapperAddress("erd1ec8xj629h55d7v7s6fz39h690vk9zs5dvdxk8lp4ct9k6vhgttuqaaj34x")
second_owner.sync_nonce(proxy)

first_owner = Account(pem_file="~/Documents/sh2.pem")
first_owner.address = WrapperAddress("erd1rjff57qeq4ehfjmm4wfz08eygf9ej0d44hs2wsjnfp7zxpuea7aq78vkhk")
first_owner.sync_nonce(proxy)

user = Account(pem_file="~/Documents/sh2.pem")
user.address = WrapperAddress("erd1qjsdgvv88zaeuqr569vvknywel7ua62nemeusldc8ls7quldl7fq3nskc6")
user.sync_nonce(proxy)

change owner

In [ ]:
hash = endpoint_call(proxy, 50000000, second_owner, contract_address, 
                     "ChangeOwnerAddress", [first_owner.address])

upgrade

In [ ]:
from multiversx_sdk import CodeMetadata
from utils.utils_tx import upgrade_call

metadata = CodeMetadata(upgradeable=True, payable_by_contract=False, readable=True)
gas_limit = 200000000
bytecode_path = Path().home() / "contract.wasm"

arguments = []

hash = upgrade_call("launchpad", proxy, gas_limit, second_owner, contract_address,
                    bytecode_path, metadata, arguments)

blocks

In [ ]:
proxy.do_post(f"{GENERATE_BLOCKS_URL}/1", {})

In [ ]:
hash = endpoint_call(proxy, 50000000, user, contract_address, "claimLaunchpadTokens", [])